In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Install and import necessary libaries

In [ ]:
# !pip install --force-reinstall --pre torch --index-url https://download.pytorch.org/whl/nightly/cu117

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops
!pip install -q -U pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━

In [5]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
import json
import pandas as pd
import os
import ast
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from pprint import pprint
from tqdm import tqdm
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
print(torch.__version__)

2.3.0+cu121


# 2. Sign in to huggingface

In [8]:
notebook_login()

# 3. Load pretrained LLM

In [9]:
MODEL_NAME = "deepseek-ai/deepseek-math-7b-rl"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.8k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
    )

In [11]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [12]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 37478400 || all params: 3912216576 || trainables%: 0.9579837739535205


# 4. Test pretrained model performance (make prediction)

In [13]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [14]:
prompt = """
<|im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
### Câu hỏi:
Số gồm 1 đơn vị và 3 chục đọc là :
### Các lựa chọn:
A. 30
B. 31
C. 20
D. 21
### Câu trả lời:

<|im_start|>assistant
""".strip()

In [15]:
%%time
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'''))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:538: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<|im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
### Câu hỏi:
Số gồm 1 đơn vị và 3 chục đọc là :
### Các lựa chọn:
A. 30
B. 31
C. 20
D. 21
### Câu trả lời:

<|im_start|>assistant
Đáp án B. 31
Vì số gồm 1 đơn vị và 3 chục đọc là 31.
The answer is $\boxed{B}$.
CPU times: user 11.2 s, sys: 1.06 s, total: 12.3 s
Wall time: 16.7 s


# 5. Fine-tuning LLM

### 5.1. Prepare dataset ZaLo

In [16]:
def generate_prompt_train(question, choices, explanation, answer):

    answer_option = answer.split('. ')[0]  # Extracting only the answer option letter (e.g., "D")

    formatted_answer = f"$\\boxed{{{answer_option}}}$"

    return f"""
    <|im_start|>system
    You are an expert in math. You will receive multiple choice questions with options, solve step by step if available and choose the correct option.

    <|im_start|>user
    ### Question:
    {question}
    ### Choices:
    {choices}\nPlease reason step by step, and put your final answer within \\boxed{{}}.
    ### Answer:

    <|im_start|>assistant
    {explanation}\nThe answer is {formatted_answer}.
    """.strip()


def generate_and_tokenize_prompt(tokenizer, question, choices, explanation, answer):
    full_prompt = generate_prompt_train(question, choices, explanation, answer)
    tokenized_full_prompt = tokenizer(
        full_prompt,
        padding=True,
        truncation=True
    )

    return tokenized_full_prompt


def process_data_train(file_path, tokenizer):

    print(f'has data file: {os.path.isfile(file_path)}')

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    training_samples = []
    choices_data = []

    for sample in data["data"]:
        try:
            choices = ast.literal_eval(sample['choices'])

            if sample['id'] == '1':
                print(sample['question'])

            explanation = sample['explanation'].strip()
            question = sample['question']
            answer = sample['answer']

            choices = '\n'.join(choices)
            training_sample = generate_and_tokenize_prompt(
                tokenizer, question, choices, explanation, answer
            )
        except Exception as ex:
            print(f"ID: {sample['id']} Error {ex}")

        training_samples.append(training_sample)

    choices_data = Dataset.from_list(training_samples)

    return choices_data

In [17]:
file_path = '/content/drive/MyDrive/Math-Solver/train.json'
choices_data = process_data_train(file_path, tokenizer)

has data file: True
Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính


In [18]:
choices_data

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1196
})

In [19]:
type(choices_data)

datasets.arrow_dataset.Dataset

### 5.2. Training

In [21]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=choices_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.618300
2,1.532900
3,1.601400
4,1.275600
5,1.665800
6,1.106200
7,1.386300
8,1.512200
9,1.151700
10,0.867600


TrainOutput(global_step=299, training_loss=0.28668802029711743, metrics={'train_runtime': 3118.7117, 'train_samples_per_second': 0.383, 'train_steps_per_second': 0.096, 'total_flos': 2.2889506282438656e+16, 'train_loss': 0.28668802029711743, 'epoch': 1.0})

### 5.3. Test prediction

In [22]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
You are an expert in math. You will receive multiple choice questions with options, solve step by step if available and choose the correct option.

<|im_start|>user
### Question:
Số gồm 1 đơn vị và 2 chục đọc là :
### Choices:
A. 20
B. 21
C. 30
D. 31
### Answer:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:538: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the 

<|im_start|>system
You are an expert in math. You will receive multiple choice questions with options, solve step by step if available and choose the correct option.

<|im_start|>user
### Question:
Số gồm 1 đơn vị và 2 chục đọc là :
### Choices:
A. 20
B. 21
C. 30
D. 31
### Answer:

<|im_start|>assistant
Số gồm 1 đơn vị và 2 chục đọc là : Hai mươi mốt.
The answer is $\boxed{B}$.
CPU times: user 39.4 s, sys: 17.7 s, total: 57.1 s
Wall time: 58.5 s


# 6. Save model to huggingface

In [23]:
model.save_pretrained("trained-model")

In [25]:
notebook_login()

In [26]:
PEFT_MODEL = "datnth1709/deepseek-math-7b-zaloai2023"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/150M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datnth1709/deepseek-math-7b-zaloai2023/commit/5b46b6197e98358c4f93efa6bb4e015838ccf425', commit_message='Upload model', commit_description='', oid='5b46b6197e98358c4f93efa6bb4e015838ccf425', pr_url=None, pr_revision=None, pr_num=None)

# 7. Inference

In [27]:
PEFT_MODEL = "datnth1709/deepseek-math-7b-zaloai2023"

config = PeftConfig.from_pretrained(PEFT_MODEL)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/150M [00:00<?, ?B/s]

In [28]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = model.config.pad_token_id
generation_config.eos_token_id = model.config.eos_token_id

In [29]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = f"""
<|im_start|>system
You are an expert in math. You will receive multiple choice questions with options, solve step by step if available and choose the correct option.

<|im_start|>user
### Question:
Một hình bình hành có độ dày đáy bằng $\\frac{{3}}{{2}}$ dm và chiều cao bằng $\\frac{{1}}{{2}}$ độ dài đáy. Diện tích của hình bình hành là:
### Choices:
A. $\\\\frac{{9}}{{4}}$ dm2
B. $\\\\frac{{9}}{{16}}$ dm2
C. $\\\\frac{{9}}{{8}}$ dm2
D. $\\\\frac{{3}}{{4}}$ dm2
Please reason step by step, and put your final answer within \\boxed{{}}.
### Answer:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:538: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


<|im_start|>system
You are an expert in math. You will receive multiple choice questions with options, solve step by step if available and choose the correct option.

<|im_start|>user
### Question:
Một hình bình hành có độ dày đáy bằng $\frac{3}{2}$ dm và chiều cao bằng $\frac{1}{2}$ độ dài đáy. Diện tích của hình bình hành là:
### Choices:
A. $\\frac{9}{4}$ dm2
B. $\\frac{9}{16}$ dm2
C. $\\frac{9}{8}$ dm2
D. $\\frac{3}{4}$ dm2
Please reason step by step, and put your final answer within \boxed{}.
### Answer:

<|im_start|>assistant
Để giải quyết câu hỏi này, chúng ta cần hiểu rõ về công thức tính diện tích của một hình bình hành. Diện tích của một hình bình hành được tính bằng cách nhân độ dài của đáy với chiều cao của nó.

Trong trường hợp này, độ dài của đáy là $\frac{3}{2}$ dm và chiều cao
CPU times: user 31.4 s, sys: 642 ms, total: 32.1 s
Wall time: 35.4 s


In [30]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = f"""
<|im_start|>system
You are an expert in math. You will receive multiple choice questions with options, solve step by step if available and choose the correct option.

<|im_start|>user
### Question:
Tìm một số, biết rằng nếu lấy số đó bớt đi 3454 rồi gấp lên 3 lần cuối cùng thêm 100 vào thì được 1036.
### Choices:
A. 1036
B. 3454
C. 312
D. 3766
Please reason step by step, and put your final answer within \\boxed{{}}.
### Answer:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


<|im_start|>system
You are an expert in math. You will receive multiple choice questions with options, solve step by step if available and choose the correct option.

<|im_start|>user
### Question:
Tìm một số, biết rằng nếu lấy số đó bớt đi 3454 rồi gấp lên 3 lần cuối cùng thêm 100 vào thì được 1036.
### Choices:
A. 1036
B. 3454
C. 312
D. 3766
Please reason step by step, and put your final answer within \boxed{}.
### Answer:

<|im_start|>assistant
Đầu tiên, chúng ta cần hiểu rằng câu hỏi đang yêu cầu chúng ta tìm một số, sau đó lấy số đó trừ đi 3454, gấp lên 3 lần và cuối cùng cộng thêm 100 để được kết quả là 1036.

Bước 1: Đặt số cần tìm là x.

Bước 2: Theo câu hỏi, chúng ta có phương
CPU times: user 28.5 s, sys: 616 ms, total: 29.2 s
Wall time: 29.4 s
